# Add Waveforms
Here we combine the experiment-specific stationXML file created in GLNN_StationXML.ipynb with the recorded AE waveforms from TranAX in a tpc5 file to initialize the experiment ASDF file.

ElSys has provided a module tpc5.py __[available on their website](https://www.elsys-instruments.com/en/support/tpc5_fileformat.php)__ for easy interpretation.

In [1]:
import tpc5 # get from ElSys, see link above
import h5py
import numpy as np
import pyasdf
from obspy import Stream, Trace
from obspy.core.util import AttribDict
from obspy.core import Stats
import obspy

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First set up the ASDF dataset to fill

In [2]:
ds = pyasdf.ASDFDataSet('020918_shear/020918ASDF.h5', compression='gzip-3')
ds.add_stationxml('020918_shear/GLNNstations_020918.xml')

In [3]:
ds.waveforms.list()

['L0.AE09',
 'L0.AE11',
 'L0.AE16',
 'L0.AE17',
 'L0.AE18',
 'L0.AE21',
 'L0.AE22',
 'L0.AE23',
 'L0.AE27',
 'L0.AE28',
 'L0.AE32',
 'L0.AE33',
 'L0.AE35',
 'L0.AE38',
 'L0.AE41',
 'L0.AE43']

In [6]:
ds.waveforms.L0_AE09.StationXML.networks[0].stations[0]
# ASDF appears to dump the extra attribute of the StationXML
# Add the (x,y) locations as auxiliary data instead

Station AE09 (BP3)
	Station Code: AE09
	Channel Count: 1/None (Selected/Total)
	None - 
	Access: None 
	Latitude: 37.87, Longitude: -122.26, Elevation: 100.0 m
	Available Channels:
		AE09.00.FHZ

The stations are ordered as A1-D4 in the original StationXML but get re-sorted to increasing station codes when imported to ASDF. The ASDF format also drops the .extra attribute from the StationXML, which contained my local_location information. I need to temporarily access the StationXML outside of ASDF to bring back this lost information.

In [34]:
inv = obspy.read_inventory('020918_shear/GLNNstations_020918.xml', format='stationxml')
ordered_stations = [inv[0].stations[i].code for i in range(16)]
ordered_locations = [(float(inv[0].stations[i].extra.local_location.value['x'].value),
                      float(inv[0].stations[i].extra.local_location.value['y'].value),
                      float(inv[0].stations[i].extra.local_location.value['z'].value))
                     for i in range(16)]


These bits of extra info are great candidates for ASDF auxiliary data.

In [22]:
ordered_locations

[(27.45232, 19.05, 0.0),
 (41.447720000000004, 19.05, 0.0),
 (34.46272, 19.05, 0.0),
 (30.957520000000002, 19.05, 0.0),
 (19.558, 16.002, 0.0),
 (20.44192, 19.05, 0.0),
 (34.29, 26.162000000000003, 0.0),
 (37.846000000000004, 22.352000000000004, 0.0),
 (30.957520000000002, 22.555200000000003, 0.0),
 (26.46172, 23.368, 0.0),
 (23.876, 22.352000000000004, 0.0),
 (13.43152, 19.05, 0.0),
 (35.45332, 14.732, 0.0),
 (30.957520000000002, 15.5448, 0.0),
 (9.9568, 19.05, 0.0),
 (23.876, 15.748000000000001, 0.0)]

In [44]:
np.array([s.encode('utf8') for s in ordered_stations])
# h5 files can't handle the numpy unicode datatype, this is a workaround

array([b'AE27', b'AE09', b'AE17', b'AE22', b'AE38', b'AE35', b'AE18',
       b'AE11', b'AE21', b'AE28', b'AE32', b'AE41', b'AE16', b'AE23',
       b'AE43', b'AE33'], dtype='|S4')

In [41]:
ds.add_auxiliary_data(data=np.array(ordered_locations),
                      data_type='LabStationInfo',
                      path='local_locations',
                      parameters={})
ds.add_auxiliary_data(data=np.array([s.encode('utf8') for s in ordered_stations]),
                      data_type='LabStationInfo',
                      path='ordered_stations',
                      parameters={})

In [60]:
ds.auxiliary_data.LabStationInfo.local_locations.data[:]

array([[27.45232, 19.05   ,  0.     ],
       [41.44772, 19.05   ,  0.     ],
       [34.46272, 19.05   ,  0.     ],
       [30.95752, 19.05   ,  0.     ],
       [19.558  , 16.002  ,  0.     ],
       [20.44192, 19.05   ,  0.     ],
       [34.29   , 26.162  ,  0.     ],
       [37.846  , 22.352  ,  0.     ],
       [30.95752, 22.5552 ,  0.     ],
       [26.46172, 23.368  ,  0.     ],
       [23.876  , 22.352  ,  0.     ],
       [13.43152, 19.05   ,  0.     ],
       [35.45332, 14.732  ,  0.     ],
       [30.95752, 15.5448 ,  0.     ],
       [ 9.9568 , 19.05   ,  0.     ],
       [23.876  , 15.748  ,  0.     ]])

Now on to importing the waveforms. An experiment might be made up of multiple files. Display the tpc5 files present in the experiment folder:

In [3]:
!ls 020918_shear/*.tpc5

020918_shear/bd1.tpc5  020918_shear/run.tpc5
020918_shear/bd2.tpc5  020918_shear/timing.tpc5


This experiment had two ball drops (bd1, bd2), a timing alignment signal (timing), and the full shear run (run). I'll start with the first ball drop.

In [61]:
f = h5py.File("020918_shear/run.tpc5", "r")

In [62]:
# we need to know how many blocks to read
# all channels have the same number of blocks, use channel 1
cg = f[tpc5.getChannelGroupName(1)]
nblocks = len(cg['blocks'].keys())

In [135]:
# read the raw data from each channel into a stream, one trace at a time
# first build some basic Stats
statn_stats = Stats()
statn_stats.network = 'L0'
statn_stats.channel = 'FHZ'
statn_stats.location = '00'
statn_stats.sampling_rate = 20e6

# make sure times will retain full precision
# the ElSys max precision seems to be nanoseconds
obspy.UTCDateTime.DEFAULT_PRECISION = 9

# iterate through stations, following the ordered_stations A1-D4 sort
# the number of the (ordered) station is the TranAX channel A1-D4->1-16, here Tchan
for Tchan,statname in enumerate(ordered_stations,1):
    # add the station to the stats
    statn_stats.station = statname
    
    # create a stream for the station
    statn_stream = Stream()
#     set_trace()
    
    # iterate through continuous data segments
    # TranAX calls these Blocks, obspy calls them Traces
    for blk in range(1,nblocks+1):
        # get the trace start time
        statn_stats.starttime = (obspy.UTCDateTime(tpc5.getStartTime(f,1)) # gives the start of the whole recording
                                + tpc5.getTriggerTime(f,1,block=blk) # seconds from start to trigger
                                - tpc5.getTriggerSample(f,1,block=blk)/statn_stats.sampling_rate) # seconds from trigger to block start
        
        # get the raw voltage data
        raw = tpc5.getVoltageData(f, Tchan, block=blk)
        # give the stats the length, otherwise it takes 0 points
        statn_stats.npts = len(raw)
        statn_stream += Trace(raw ,statn_stats)
    
    # add the complete stream to the ASDF object
    ds.add_waveforms(statn_stream, "raw_record")

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE27/L0.AE27.00.FHZ__2018-02-09T22:36:00__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE27/L0.AE27.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE27/L0.AE27.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE27/L0.AE27.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE09/L0.AE09.00.FHZ__2018-02-09T22:50:16__2018-02-09T22:50:16__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE09/L0.AE09.00.FHZ__2018-02-09T22:50:16__2018-02-09T22:50:16__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE09/L0.AE09.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE09/L0.AE09.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE22/L0.AE22.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE22/L0.AE22.00.FHZ__2018-02-09T22:46:00__2018-02-09T22:46:00__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE22/L0.AE22.00.FHZ__2018-02-09T22:46:00__2018-02-09T22:46:00__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE22/L0.AE22.00.FHZ__2018-02-09T22:46:00__2018-02-09T22:46:00__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE38/L0.AE38.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE38/L0.AE38.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE35/L0.AE35.00.FHZ__2018-02-09T22:36:00__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE35/L0.AE35.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE18/L0.AE18.00.FHZ__2018-02-09T22:50:13__2018-02-09T22:50:13__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE18/L0.AE18.00.FHZ__2018-02-09T22:50:15__2018-02-09T22:50:15__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE18/L0.AE18.00.FHZ__2018-02-09T22:50:16__2018-02-09T22:50:16__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE18/L0.AE18.00.FHZ__2018-02-09T22:50:16__2018-02-09T22:50:16__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE21/L0.AE21.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE21/L0.AE21.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE21/L0.AE21.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE21/L0.AE21.00.FHZ__2018-02-09T22:46:00__2018-02-09T22:46:00__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE28/L0.AE28.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE28/L0.AE28.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE28/L0.AE28.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE28/L0.AE28.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE41/L0.AE41.00.FHZ__2018-02-09T22:46:00__2018-02-09T22:46:00__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE41/L0.AE41.00.FHZ__2018-02-09T22:50:13__2018-02-09T22:50:13__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE41/L0.AE41.00.FHZ__2018-02-09T22:50:13__2018-02-09T22:50:13__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE41/L0.AE41.00.FHZ__2018-02-09T22:50:15__2018-02-09T22:50:15__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE23/L0.AE23.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE23/L0.AE23.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE23/L0.AE23.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE23/L0.AE23.00.FHZ__2018-02-09T22:36:01__2018-02-09T22:36:01__raw_record' alrea

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE43/L0.AE43.00.FHZ__2018-02-09T22:50:16__2018-02-09T22:50:16__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE43/L0.AE43.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE43/L0.AE43.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' already exists in file. Will not be added!
  closure_warn(self, *args, **kwargs)
/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/pyasdf/asdf_data_set.py:51: ASDFWarning: Data 'L0.AE43/L0.AE43.00.FHZ__2018-02-09T22:50:17__2018-02-09T22:50:17__raw_record' alrea

In [133]:
# [len(tpc5.getVoltageData(f,1,block=i)) for i in range(1,nblocks+1)] # length varies
statn_stats.station = ordered_stations[0]
statn_stream = Stream()
statn_stats.starttime = (obspy.UTCDateTime(tpc5.getStartTime(f,1)) # gives the start of the whole recording
                                + tpc5.getTriggerTime(f,1,block=1) # seconds from start to trigger
                                - tpc5.getTriggerSample(f,1,block=1)/statn_stats.sampling_rate) # seconds from trigger to block start
v = tpc5.getVoltageData(f,1)
statn_stats.npts = len(v)
statn_stream += Trace(v, statn_stats)

In [146]:
starttime.strftime("%Y-%m-%dT%H:%M:%S.%f")

'2018-02-09T22:27:24.123456'

In [138]:
ds.waveforms.L0_AE27.raw_record

16 Trace(s) in Stream:
L0.AE27.00.FHZ | 2018-02-09T22:30:51.423920384Z - 2018-02-09T22:30:51.460705934Z | 20000000.0 Hz, 735712 samples
L0.AE27.00.FHZ | 2018-02-09T22:30:59.677762816Z - 2018-02-09T22:30:59.710336666Z | 20000000.0 Hz, 651478 samples
L0.AE27.00.FHZ | 2018-02-09T22:31:33.355811328Z - 2018-02-09T22:31:33.398032628Z | 20000000.0 Hz, 844427 samples
L0.AE27.00.FHZ | 2018-02-09T22:32:38.097585664Z - 2018-02-09T22:32:38.142654964Z | 20000000.0 Hz, 901387 samples
L0.AE27.00.FHZ | 2018-02-09T22:36:00.631816448Z - 2018-02-09T22:36:00.664316398Z | 20000000.0 Hz, 650000 samples
L0.AE27.00.FHZ | 2018-02-09T22:36:00.976014592Z - 2018-02-09T22:36:01.008514542Z | 20000000.0 Hz, 650000 samples
L0.AE27.00.FHZ | 2018-02-09T22:36:01.172681472Z - 2018-02-09T22:36:01.205183972Z | 20000000.0 Hz, 650051 samples
L0.AE27.00.FHZ | 2018-02-09T22:45:59.536139008Z - 2018-02-09T22:45:59.568639208Z | 20000000.0 Hz, 650005 samples
L0.AE27.00.FHZ | 2018-02-09T22:46:00.544065792Z - 2018-02-09T22:46:00.576

In [114]:
ds.waveforms.L0_AE27.raw_record[0].stats

         network: L0
         station: AE27
        location: 00
         channel: FHZ
       starttime: 2018-02-09T22:30:51.423920384Z
         endtime: 2018-02-09T22:30:51.423920384Z
   sampling_rate: 20000000.0
           delta: 5e-08
            npts: 0
           calib: 1.0
         _format: ASDF
            asdf: AttribDict({'format_version': '1.0.1', 'tag': 'raw_record'})

In [82]:
from IPython.core.debugger import set_trace

## Extra notes
Accessing tpc5 and hdf5 files can be a bit confusing. Here are some reminders:

In [ ]:
# to view the keys of an hdf5 file:
list(f.keys())

# to then access one of the keys:
f['key']

# to control the precision of a UTCDateTime
t = obspy.UTCDateTime(precision=9)
# doing arithmetic creates a new UTCDateTime with the default precision!
# change the default precision for the session
obspy.UTCDateTime.DEFAULT_PRECISION = 9

In [ ]:
# clean up a botched attempt at adding raw_records
[ds.waveforms['L0_'+code].__delitem__('raw_record') for code in ordered_stations]